In [ ]:
pip install scikeras, keras-tuner

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import load_iris
import keras_tuner as kt
import matplotlib.pyplot as plt
import seaborn as sns

# Load the IRIS dataset
data = load_iris()
X = data.data  # Features
y = data.target  # Labels

# Encode labels to one-hot encoding
encoder = LabelEncoder()
y = encoder.fit_transform(y)
y = to_categorical(y)

# Perform Exploratory Data Analysis (EDA)
plt.figure(figsize=(8, 6))
sns.boxplot(data=X)
plt.title("Feature Distribution of IRIS Dataset")
plt.show()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features for better convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model-building function for hyperparameter tuning
def build_model(hp):
    model = Sequential()

    # Input layer with number of neurons as a hyperparameter
    model.add(Dense(units=hp.Int('units_input', min_value=8, max_value=64, step=8),
                    activation='relu', input_shape=(X_train.shape[1],)))

    # Hidden layer with tunable neurons
    model.add(Dense(units=hp.Int('units_hidden', min_value=8, max_value=64, step=8),
                    activation='relu'))

    # Dropout layer to reduce overfitting
    model.add(Dropout(rate=hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)))

    # Output layer (since IRIS dataset has 3 classes, we use softmax activation)
    model.add(Dense(3, activation='softmax'))

    # Compile the model with tunable learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Float('learning_rate',
                                                                         min_value=0.001,
                                                                         max_value=0.01,
                                                                         step=0.001)),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Initialize the Keras tuner with Hyperband search strategy
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='hyper_tuning',
                     project_name='iris_ann')

# Define early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Perform the hyperparameter search
tuner.search(X_train, y_train, validation_data=(X_test, y_test), epochs=20, callbacks=[early_stopping])

# Get the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Optimal Hyperparameters:\n"
      f"Input Layer Units: {best_hps.get('units_input')}\n"
      f"Hidden Layer Units: {best_hps.get('units_hidden')}\n"
      f"Dropout Rate: {best_hps.get('dropout_rate')}\n"
      f"Learning Rate: {best_hps.get('learning_rate')}")

# Build and train the best model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, callbacks=[early_stopping])

# Evaluate the final model
test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy:.4f}')
print(f'Test Loss: {test_loss:.4f}')

# Test the model by making predictions
predictions = best_model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)  # Convert to class labels

# Convert the one-hot encoded y_test back to class labels
true_classes = np.argmax(y_test, axis=1)

# Display some predictions vs true labels
print("\nPredictions vs True Labels:")
for i in range(10):  # Display first 10 predictions
    print(f"Predicted: {predicted_classes[i]}, True: {true_classes[i]}")

# Plot training metrics
def plot_metrics(history):
    plt.figure(figsize=(12, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()

    plt.show()

plot_metrics(history)

The model achieved **100% test accuracy** with a **test loss of 0.0413**, indicating excellent performance on the test set. The **optimal hyperparameters** for the model were:

- **Input Layer Units**: 32
- **Hidden Layer Units**: 64
- **Dropout Rate**: 0.2
- **Learning Rate**: 0.006

The predictions made by the model on the test set matched the true labels perfectly for the first 10 test samples, showing the model's ability to classify correctly across different classes (0, 1, and 2) in the Iris dataset.

Code - Explaination

Import Required Libraries:

Imports necessary libraries such as TensorFlow, Keras, scikit-learn, and Matplotlib for model building, data preprocessing, and visualization.

Load IRIS Dataset:

Loads the IRIS dataset using load_iris() from sklearn.datasets and assigns features (X) and labels (y).

Preprocess Labels:

Encodes labels into one-hot encoding using LabelEncoder and to_categorical for multi-class classification.

Exploratory Data Analysis (EDA):

Visualizes feature distributions of the IRIS dataset using a boxplot for data inspection.

Train-Test Split:

Splits the dataset into training (80%) and testing (20%) sets using train_test_split from sklearn.

Feature Normalization:

Normalizes the features using StandardScaler to improve model convergence during training.

Model Definition for Hyperparameter Tuning:

Defines a function build_model that takes hyperparameters as input. It builds a neural network with:

An input layer with tunable neurons.

A hidden layer with tunable neurons.

A dropout layer for regularization.

An output layer with softmax activation for multi-class classification.

Hyperparameter Search Initialization:

Initializes Keras Tuner with Hyperband strategy to perform hyperparameter optimization over specified ranges.

Early Stopping Callback:

Defines an early stopping callback to prevent overfitting, halting training when validation loss does not improve.

Hyperparameter Search:

Searches for the best hyperparameters using the tuner.search() method, based on validation accuracy.

Retrieve Best Hyperparameters:

After tuning, retrieves the optimal hyperparameters for the best model.

Model Training:

Builds and trains the model using the best-found hyperparameters, with the early stopping callback to monitor validation loss.

Model Evaluation:

Evaluates the model on the test set and prints the test accuracy and loss.

Predictions and True Labels Comparison:

Makes predictions on the test set, converts the predictions from one-hot encoding to class labels, and compares the predictions with true labels.

Metrics Visualization:

Plots the training and validation accuracy and loss across epochs to visualize model performance.